In [ ]:
import getpass
import os

os.environ["OPENAI_API_KEY"] = getpass.getpass()

··········


## Modelos

In [ ]:
!pip install langchain_openai

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.4/50.4 kB 2.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 52.0/52.0 kB 3.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 396.2/396.2 kB 14.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 39.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 318.9/318.9 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.7/150.7 kB 10.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 11.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 3.6 MB/s eta 0:00:00
  Attempting uninstall: tenacity
    Found existing installation: tenacity 9.0.0
    Uninstalling tenacity-9.0.0:
      Succes

In [ ]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

In [ ]:
response = llm.invoke("Olá")

print(response)

content='Olá! Como posso ajudá-lo hoje?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 8, 'prompt_tokens': 8, 'total_tokens': 16}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None} id='run-466aa49d-255a-4917-ae55-8b7fb3d3d492-0' usage_metadata={'input_tokens': 8, 'output_tokens': 8, 'total_tokens': 16}


In [ ]:
response.content

'Olá! Como posso ajudá-lo hoje?'

## Prompts

### Templates Simples

In [ ]:
from langchain_core.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate.from_template("Traduza o seguinte texto para português: {texto}")

#Runnable
prompt = prompt_template.invoke({"texto": "Artificial Intelligence is the future!"})
print(prompt)

messages=[HumanMessage(content='Traduza o seguinte texto para português: Artificial Intelligence is the future!')]


In [ ]:
response = llm.invoke(prompt)

print(response.content)

Inteligência Artificial é o futuro!


### Templates de Mensagens

In [ ]:
from langchain_core.messages import SystemMessage, HumanMessage

messages = [
    SystemMessage(content="Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas."),
    HumanMessage(content="Hello, how are you?"),
]

#messages = [
#     ("system", "Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas."),
#     ("human", "Hello, how are you?"),
# ]

response = llm.invoke(messages)

print(response)

content='Olá, como você está?' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 6, 'prompt_tokens': 36, 'total_tokens': 42}, 'model_name': 'gpt-4o-mini-2024-07-18', 'system_fingerprint': 'fp_f33667828e', 'finish_reason': 'stop', 'logprobs': None} id='run-7ca1b793-42d4-4b51-8514-759d014f3c47-0' usage_metadata={'input_tokens': 36, 'output_tokens': 6, 'total_tokens': 42}


In [ ]:
prompt_template = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um tradutor de {lingua_origem} para {lingua_destino}. Traduza as mensagens que forem enviadas."),
        ("user", "{texto}")
    ]
)

In [ ]:
prompt = prompt_template.invoke({
    "lingua_origem": "inglês",
    "lingua_destino": "português",
    "texto": "Hello, how are you?"
})

print(prompt)

messages=[SystemMessage(content='Você é um tradutor de inglês para português. Traduza as mensagens que forem enviadas.'), HumanMessage(content='Hello, how are you?')]


In [ ]:
response = llm.invoke(prompt)

print(response.content)

Olá, como você está?


### Parsers

In [ ]:
from langchain_core.output_parsers import StrOutputParser

parser = StrOutputParser()

In [ ]:
output = parser.invoke(response)

print(output)

Olá, como você está?


## Encadeamento

In [ ]:
chain = prompt_template | llm | parser

In [ ]:
response = chain.invoke({
    "lingua_origem": "inglês",
    "lingua_destino": "espanhol",
    "texto": "As praias de Recife tem tubarões!"
})

print(response)

¡Las playas de Recife tienen tiburones!


In [ ]:
def translate(texto, lingua_origem, lingua_destino):
    response = chain.invoke({
        "lingua_origem": lingua_origem,
        "lingua_destino": lingua_destino,
        "texto": texto
    })
    return response

In [ ]:
output = translate("The beaches of Recife have sharks!", "inglês", "espanhol")

print(output)

¡Las playas de Recife tienen tiburones!


## Exercícios

### Exercício 1
Crie uma `chain` que a partir de um tópico informado pelo usuário, crie uma piada.

In [ ]:
prompt_template_piada = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um comediante e conta piadas sobre diversos temas. Conte uma piada sobre {tema}."),
        ("user", "{tema}")
    ]
)

chain_piada = prompt_template_piada | llm | parser

response = chain_piada.invoke({
    "tema": "praia"
})

print(response)

Por que o peixe não gosta de ir à praia?

Porque ele tem medo de ser pescado! 🐟🏖️


### Exercício 2
Crie uma `chain` que classifique o sentimento de um texto de entrada em positivo, neutro ou negativo.

In [ ]:
prompt_template_sentimento = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um especialista em sentimento e precisa classificar o texto a seguir em positivo, neutro ou negativo: {texto}."),
        ("user", "{texto}")
    ]
)

chain_sentimento = prompt_template_sentimento | llm | parser

response = chain_sentimento.invoke({
    "texto": "Eu estou muito estressado"
})

print(response)

O sentimento do texto é negativo.


### Exercício 3
Crie uma `chain` que gere o código de uma função Python de acordo com a descrição do usuário.

In [ ]:
prompt_template_codigo = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um especialista em Python e precisa gerar o código de uma função Python de acordo com a descrição do usuário: {descricao}."),
        ("user", "{descricao}")
    ]
)

chain_codigo = prompt_template_codigo | llm | parser

response = chain_codigo.invoke({
    "descricao": "Uma função que verifica se um número é par"
})

print(response)

Claro! Aqui está uma função em Python que verifica se um número é par:

```python
def is_even(number):
    """Verifica se um número é par.
    
    Args:
        number (int): O número a ser verificado.
        
    Returns:
        bool: True se o número for par, False caso contrário.
    """
    return number % 2 == 0

# Exemplo de uso
num = 4
if is_even(num):
    print(f"{num} é par.")
else:
    print(f"{num} é ímpar.")
```

Essa função `is_even` recebe um número como argumento e retorna `True` se o número for par (ou seja, se o resto da divisão por 2 for igual a 0) e `False` caso contrário. Você pode testar a função com diferentes números para verificar seu funcionamento!


### Exercício 4
Crie uma `chain` que explique de forma simplificada um tópico geral fornecido pelo usuário e, em seguida, traduza a explicação para inglês. Utilize dois templates encadeados.

In [ ]:
prompt_template_explicacao = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é um especialista em explicações de diversos temas. Explique de forma simplificada o seguinte tópico: {topico}."),
        ("user", "{topico}")
    ]
)

chain_explicacao = prompt_template_explicacao | llm | parser

response = chain_explicacao.invoke({
    "topico": "Universidades publicas"
})

print(response)

Universidades públicas são instituições de ensino superior que são mantidas e financiadas pelo governo. Isso significa que elas recebem recursos do Estado, o que permite que ofereçam cursos e programas de graduação e pós-graduação sem cobrar mensalidades ou com taxas muito reduzidas para os alunos.

Aqui estão alguns pontos importantes sobre universidades públicas:

1. **Acesso**: O objetivo das universidades públicas é proporcionar acesso à educação superior para um maior número de pessoas, independentemente de sua condição financeira.

2. **Financiamento**: O governo investe dinheiro nessas instituições, o que ajuda a cobrir custos como salários de professores, manutenção das instalações e recursos educacionais.

3. **Qualidade**: Muitas universidades públicas são reconhecidas por sua qualidade acadêmica e têm programas de pesquisa relevantes. Elas costumam contar com professores qualificados e infraestrutura adequada.

4. **Concursos e Vagas**: O ingresso nas universidades públicas 

### Exercício 5 - Desafio
Crie uma `chain` que responda perguntas sobre o CESAR School.

In [ ]:
prompt_template_cesar = ChatPromptTemplate.from_messages(
    [
        ("system", "Você é trabalha no CESAR School, uma instituição de ensino que está localizada em Recife-PE com formações de graduação, especialização, mestrado e doutorado na área de tecnologia, inovação e gestão. Responda as perguntas do usuario. {perguntas}"),
        ("user", "{perguntas}")]
)

chain_cesar = prompt_template_cesar | llm | parser

response = chain_cesar.invoke({
    "perguntas": "Quem é o CEO do CESAR School?"
})

print(response)

O CEO do CESAR School é o professor e empreendedor Silvio Meira. Ele é conhecido por sua contribuição significativa para a área de tecnologia e inovação no Brasil. Se precisar de mais informações sobre o CESAR School ou suas formações, fique à vontade para perguntar!
